In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os 
import seaborn as sns
import matplotlib.pyplot as plt
import requests

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
inputdir = '/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset/simpsons_dataset'


[Google Drive: saved_model](https://drive.google.com/file/d/1GgTxWW6Hfy6VVyw-d216aHg-958hwgLP/view?usp=sharing)</br>
[the-simpsons-characters-dataset](https://www.kaggle.com/alexattia/the-simpsons-characters-dataset)

In [2]:
#directory check
# for root,_,_ in os.walk('/kaggle/input'):
#     print(root)

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
import random

In [4]:
simpson_char_list = [i for i in os.listdir(inputdir)]
print(f'simpson character : {len(simpson_char_list)}')
print(f'simson characer list\n{simpson_char_list}')

In [5]:
def img_show(img_path,img_size=(512,512)):
    import cv2
    image = cv2.imread(img_path)
    image = cv2.resize(image,img_size)
    plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(img_path.split('/')[-2])

In [6]:
img_show('../input/the-simpsons-characters-dataset/characters_illustration.png')

In [7]:
#show sample picture
img_path = os.path.join(inputdir,simpson_char_list[0], 'pic_0022.jpg')
img_show(img_path = img_path)

In [8]:
# show every character
def all_char_image_show(img_path, figsize=(12,10)):
    fig = plt.figure(figsize=figsize)
#    fig.tight_layout()
    col = int(np.floor(np.sqrt(len(simpson_char_list))))
    row = int(np.ceil(len(simpson_char_list) / col))
    
    first_img_path = []
    for root,dirs,files in os.walk(img_path):
        if root == inputdir: continue
        first_img_path.append(os.path.join(root,files[0]))
    axes=[]
    for a in range(len(first_img_path)):
        axes.append(fig.add_subplot(row,col,a+1))
        img_show(first_img_path[a])
    fig.tight_layout()    
    plt.show()

all_char_image_show(inputdir)

In [9]:
def show_files_cnt_by_char(sorting=False) -> dict:    
    files_by_char = {}
    for root,dirs,files in os.walk(inputdir):
        if root == inputdir: continue
        character = root.split('/')[-1]
#             print(f"{character}: {len(files)}")
        files_by_char[character] = len(files)
    if sorting:
        sort_orders = dict(sorted(files_by_char.items(), key=lambda x: x[1]))
#         for i in sort_orders:
#             print(i[0], i[1])
        return sort_orders
    else:
        return files_by_char

In [10]:
x, y = zip(*(show_files_cnt_by_char(True).items())) # unpack a list of pairs into two tuples
fig = plt.figure(figsize=(20,10))
plt.bar(x, y)
plt.show()

In [11]:
char_data_over_200 = { key:value for (key,value) in show_files_cnt_by_char(True).items() if value >200}
print(f'characters having dataset over 200 \n{char_data_over_200}')


In [12]:
## isolation character trainset having over 200 
import shutil
import re

def copy_image_data_2_WorkingDIR():
    if not os.path.exists('/kaggle/working' + '/train_data_flow_dir'):
        os.mkdir('/kaggle/working' + '/train_data_flow_dir')
    
    # copy only characters having image data over 200
    for i in char_data_over_200.keys():
        des = '/kaggle/working' + '/train_data_flow_dir' + '/'+i
        src = inputdir + '/'+i
        print(src,des)
    #     if not os.path.exists(des):
    #         os.mkdir(des)
        #print('/kaggle/working' + '/train_data_flow_dir/' +i)
        try:
            shutil.copytree(src,des)
        except Exception as e:
            pass
copy_image_data_2_WorkingDIR()

In [13]:
TRAIN_DATA_DIR = '/kaggle/working/train_data_flow_dir'
TEST_DATA_DIR = "../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset"
BATCH_SIZE = 32
IMAGE_SIZE = (64,64)
train_gen = ImageDataGenerator(horizontal_flip=True, 
                               vertical_flip=False,
                               rotation_range=20,
                               validation_split=0.2,
                               rescale=1/255.)
train_flow_gen = train_gen.flow_from_directory(directory = TRAIN_DATA_DIR,
                             target_size = IMAGE_SIZE,
                             class_mode = 'categorical', shuffle=True,
                            subset='training',
                             batch_size=BATCH_SIZE)
valid_flow_gen = train_gen.flow_from_directory(directory = TRAIN_DATA_DIR,
                             target_size = IMAGE_SIZE,
                             class_mode = 'categorical', shuffle=True,
                            subset='validation',
                             batch_size=BATCH_SIZE)

def get_test_df():
    """
    im_name: image_name in test dir
    y: target character each image
    path: absolute path
    """
    df_imname = pd.DataFrame(os.listdir(TEST_DATA_DIR),columns=['im_name'])
    df_imname['y'] = df_imname['im_name'].apply(lambda x: re.split('_\d+',x)[0])
    df_imname['path'] = [f'{TEST_DATA_DIR}/{i}'for i in os.listdir(TEST_DATA_DIR)]
    return df_imname


test_gen = ImageDataGenerator(rescale=1/255.)
all_train_data_flow = test_gen.flow_from_directory(TRAIN_DATA_DIR,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=False,
                                                   class_mode='categorical',
                                                  batch_size=BATCH_SIZE)
test_flow_gen = test_gen.flow_from_dataframe( get_test_df() , x_col='path',y_col='y',
                             target_size = IMAGE_SIZE,
                             class_mode = 'categorical', shuffle=False,
                             batch_size=BATCH_SIZE)

In [14]:
def load_train_set(characters_name):
    # restricted by memory
    dataset_dir_list = [inputdir+'/'+i for i in characters_name]
    x_train = []
    y_train = []
    for i in dataset_dir_list:
        char_name = i.split('/')[-1]
        print('loading ',char_name)
        for j in os.listdir(i):
            x_train.append(cv2.resize(cv2.imread(i+'/'+j),(64,64)))
            y_train.append(char_name)

    return np.array(x_train), np.array(x_train)

In [15]:
# characters having over 200 dataset is same with characters in test dataset
def check_train_test_characters():
    for root,dirs,files in os.walk(TEST_DATA_DIR):
        print(len([k for k in files if any(i in k for i in char_data_over_200.keys())]))

    test_data_char_names = ''
    for root,dirs,files in os.walk(TEST_DATA_DIR):
        print(len(files))
        test_data_char_names = set([re.split('_\d+',file)[0] for file in files])
        print(test_data_char_names)

    print( len(set(char_data_over_200.keys()).union(test_data_char_names)))
    #print(f'char_names({len(set(char_data_over_200.keys()))}): {set(char_data_over_200.keys())}')
    #print(f'test_data_char_names({len(test_data_char_names)}):{test_data_char_names}')

## First Trained Model valid, test accuracy score

In [17]:
# saved model load
saved_model = tf.keras.models.load_model('../input/trained-model/simpson_recog_1.hdf5')

In [18]:
train_acc = saved_model.evaluate(all_train_data_flow, return_dict=True)
print(f'train_evaluation score: {train_acc}')

In [19]:
def show_model_score(y_true,y_pred, labels, data = 'training'):
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
    disp=ConfusionMatrixDisplay(confusion_matrix=cm)#, display_labels=labels)
    fig, ax = plt.subplots(figsize=(10, 10))
    disp.plot(ax=ax,xticks_rotation='vertical',cmap=plt.cm.Blues)
    print(f'model score on {data} data set\n')
    print(classification_report(y_true, y_pred,target_names=labels))
    plt.show()

In [22]:
y_train_pred = saved_model.predict(all_train_data_flow)
show_model_score(y_true=all_train_data_flow.classes,
                y_pred=y_train_pred.argmax(axis=1),
                labels = list(all_train_data_flow.class_indices.keys()))    

In [23]:
test_acc = saved_model.evaluate(test_flow_gen, return_dict=True)
print(f'test evaluation score: {test_acc}')

In [24]:
y_test_pred = saved_model.predict(test_flow_gen)
show_model_score(y_true=test_flow_gen.classes,
                y_pred=y_test_pred.argmax(axis=1),
                labels = list(test_flow_gen.class_indices.keys()),
                data='Testing')    

# MODEL Addtional Fitting apply color jitter
- saved

In [25]:
def color(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    import random
    if random.randint(0,10) > 4:
        x = cv2.cvtColor(x,cv2.COLOR_RGB2BGR)
    
    return x
train_gen_with_colorjitter = ImageDataGenerator(horizontal_flip=True, 
                               vertical_flip=False,
                               rotation_range=20,
                               validation_split=0.2,
                               rescale=1/255.,preprocessing_function=color,zoom_range=0.2)

colorAug_train = train_gen_with_colorjitter.flow_from_directory(TRAIN_DATA_DIR,
                                                          target_size=IMAGE_SIZE,
                                                          shuffle=True,
                                                          class_mode='categorical',
                                                          subset='training',
                                                          batch_size=BATCH_SIZE)
colorAug_val = train_gen_with_colorjitter.flow_from_directory(TRAIN_DATA_DIR,
                                                          target_size=IMAGE_SIZE,
                                                          shuffle=True,
                                                          class_mode='categorical',
                                                          subset='validation',
                                                          batch_size=BATCH_SIZE)

colorAug_noShuffle = train_gen_with_colorjitter.flow_from_directory(TRAIN_DATA_DIR,
                                                          target_size=IMAGE_SIZE,
                                                          shuffle=False,
                                                          class_mode='categorical',
                                                          subset='validation',
                                                          batch_size=BATCH_SIZE)
colorAug_test = train_gen_with_colorjitter.flow_from_dataframe(get_test_df() , x_col='path',y_col='y',
                                                               target_size = IMAGE_SIZE,
                                                               class_mode = 'categorical', shuffle=False,
                                                               batch_size=BATCH_SIZE)

In [26]:
# could it be trained, image data applied color jitter
def Model_addtional_fitting():
    saved_model = tf.keras.models.load_model('../input/simpson-model/weights-improvement-13-0.84.hdf5')
    checkpoint_path = '/kaggle/working/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    result = saved_model.fit_generator(colorAug_train, steps_per_epoch= colorAug.samples/32,validation_data=colorAug_val,
                                  epochs=10,
                                  callbacks=[checkpoint])
    return saved_model

#model_add_colorAug = Model_additonal_fitting()

In [38]:
# color aug model accuracy score
model_add_colorAug = tf.keras.models.load_model('../input/trained-model/simpson_recog_2.hdf5')
test_acc = model_add_colorAug.evaluate(colorAug_noShuffle, return_dict=True)
print(f'test evaluation score: {test_acc}')

y_test_pred = model_add_colorAug.predict(colorAug_test) # color aug model predict test data set
#print confusion matrix
show_model_score(y_true=colorAug_test.classes,
                y_pred=y_test_pred.argmax(axis=1),
                labels = list(colorAug_test.class_indices.keys()),
                data='Testing')    

In [39]:
# a,b = next(test_flow_gen)
# print(saved_model.predict(a).argmax(axis=1))
# print(b.argmax(axis=1))

def draw_images(a,b):
    a,b = next(test_flow_gen)
    for i in range(len(a)):
        plt.imshow(a[i])
        print(b[i].argmax())
        class_indices = dict({(v,k) for k,v in test_flow_gen.class_indices.items()})
        print(class_indices[b[i].argmax()])
        plt.show()
        yield
# next(draw_images(a,b))

In [40]:
def image_prediction(model, image_path, class_indices,cvt_color=True):
    image = cv2.imread(image_path)
#    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    if cvt_color: image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(64,64))
    image = np.expand_dims(image,axis=0)
    image = image/255.
    y_pred = model.predict(image).argmax(axis=-1)
#     print(image.shape)
#     print(image.max())
#     print(y_pred)
    class_indices = dict({(v,k) for k,v in class_indices.items()})
    return class_indices[y_pred[0]]
    #return list(class_indices.keys())[y_pred[0]]

def all_char_image_show_with_prediction(img_path,char_list,model):
    fig = plt.figure(figsize=(15,15))
#    fig.tight_layout()
    col = int(np.floor(np.sqrt(len(char_list))))
    row = int(np.ceil(len(char_list) / col))
    
    first_img_path = []
    for root,dirs,files in os.walk(img_path):
        if root == TRAIN_DATA_DIR: continue
        first_img_path.append(os.path.join(root,files[random.randint(0,10)]))
    axes=[]
    for a in range(len(first_img_path)):
        axes.append(fig.add_subplot(row,col,a+1))
        img_show(first_img_path[a])
        retitle = 'true: ' +first_img_path[a].split('/')[-2] + '\n' + \
        'prediction: '+image_prediction(model, first_img_path[a], test_flow_gen.class_indices, cvt_color=False)
        plt.title(retitle)
    fig.tight_layout()
    plt.show()

##### prediction when image color converted on first trained model
##### when image's color converted, first trained model cannot predict correct class not at all

In [41]:
all_char_image_show_with_prediction(TRAIN_DATA_DIR,char_data_over_200,saved_model)

##### After training with color jitter augmentation
##### predict much well

In [42]:
all_char_image_show_with_prediction(TRAIN_DATA_DIR,char_data_over_200,model_add_colorAug)

# Define Model Structure and fitting
- finished fitting and save

In [ ]:
def defCNN(num_classes):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(64,64,3)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Conv2D(32, (3, 3)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same'))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Conv2D(64, (3, 3)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding='same')) 
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

def cnnModelFitSave():
    model = defCNN(20)
    model.summary()

    checkpoint_path = '/kaggle/working/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    result = model.fit_generator(train_flow_gen,
                        steps_per_epoch=train_flow_gen.samples / 32, #32 = batch_size
                        epochs=100,
                        validation_data=valid_flow_gen, 
                        validation_freq=1,
                        callbacks =[checkpoint])

    plt.plot(result.history['loss'],label='loss')
    plt.plot(result.history['accuracy'],label='acc')
    plt.plot(result.history['val_accuracy'],label='val_acc')
    plt.plot(result.history['val_loss'],label='val_loss')
    plt.legend(loc='upper right')
    return result
    

In [127]:
saved_model.summary()